In [5]:
import networkx as nx
import matplotlib.pyplot as plt

# Replace 'yourfile.net' with the path to your actual .net file
net_file = 'marvel.net'

# Reading the .net file
G = nx.read_gexf('hero.gephi')



ParseError: not well-formed (invalid token): line 1, column 2 (<string>)